# PyTorch Model Understanding with Captum: Australian Tourism Image Analysis

This notebook demonstrates **Captum**, PyTorch's open-source library for model interpretability, using Australian tourism imagery and multilingual examples. Learn how to understand and explain your PyTorch models' behavior through various attribution techniques.

## Learning Objectives
- Understand core Captum concepts: Feature, Layer, and Neuron Attribution
- Implement **Integrated Gradients** for identifying important input features
- Use **Occlusion** analysis for perturbation-based explanations
- Apply **Grad-CAM** for layer-level interpretability
- Create interactive visualizations with **Captum Insights**
- Analyze Australian tourism images and multilingual content

## Australian Context Examples
We'll analyze images and content related to:
- 🏛️ Sydney Opera House and Harbour Bridge
- 🏖️ Gold Coast beaches and tourism
- 🐨 Australian wildlife (cats, native animals)
- 🗣️ English-Vietnamese tourism descriptions

**Captum Documentation**: https://captum.ai

---

## 1. Environment Setup and Runtime Detection

Following PyTorch best practices for cross-platform compatibility:

In [ ]:
# Environment Detection and Setup
import sys
import subprocess
import os
import time

# Detect the runtime environment
IS_COLAB = "google.colab" in sys.modules
IS_KAGGLE = "kaggle_secrets" in sys.modules or "kaggle" in os.environ.get('KAGGLE_URL_BASE', '')
IS_LOCAL = not (IS_COLAB or IS_KAGGLE)

print(f"🌐 Environment detected:")
print(f"  - Local: {IS_LOCAL}")
print(f"  - Google Colab: {IS_COLAB}")
print(f"  - Kaggle: {IS_KAGGLE}")

# Platform-specific system setup
if IS_COLAB:
    print("\n🔧 Setting up Google Colab environment...")
    # Colab usually has PyTorch pre-installed
elif IS_KAGGLE:
    print("\n🔧 Setting up Kaggle environment...")
    # Kaggle usually has most packages pre-installed
else:
    print("\n🔧 Setting up local environment...")

In [ ]:
# Install required packages based on platform
required_packages = [
    "torch",
    "torchvision", 
    "captum",
    "matplotlib",
    "seaborn",
    "numpy",
    "pandas",
    "tensorboard",
    "tqdm"
]

print("📦 Installing required packages...")
for package in required_packages:
    if IS_COLAB or IS_KAGGLE:
        # Use IPython magic commands for notebook environments
        try:
            exec(f"!pip install -q {package}")
            print(f"✅ {package}")
        except:
            print(f"⚠️ {package} (may already be installed)")
    else:
        try:
            subprocess.run([sys.executable, "-m", "pip", "install", "-q", package], 
                          capture_output=True, check=True)
            print(f"✅ {package}")
        except subprocess.CalledProcessError:
            print(f"⚠️ {package} (may already be installed)")

print("\n🎉 Package installation completed!")

In [ ]:
# Verify PyTorch and Captum installation
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter

import torchvision
import torchvision.transforms as transforms
import torchvision.models as models

# Captum imports
import captum
from captum.attr import (
    IntegratedGradients,
    Occlusion,
    LayerGradCam,
    LayerAttribution
)
from captum.attr import visualization as viz

# Additional libraries
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import pandas as pd
import seaborn as sns
from datetime import datetime
import tempfile
import json
from PIL import Image
import warnings
warnings.filterwarnings('ignore')

print(f"🔥 PyTorch {torch.__version__} ready!")
print(f"🎯 Captum {captum.__version__} ready!")
print(f"🖥️ CUDA available: {torch.cuda.is_available()}")
print(f"🎯 Device: {'GPU' if torch.cuda.is_available() else 'CPU'}")